<a href="https://colab.research.google.com/github/DuncanFerguson/GITHUB_REPOSITORY/blob/main/Project_02_Duncan_Ferguson_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import re
import os

# Importing Sklearn libraries
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

# nltk Libraries
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

!pip install pyprind
import pyprind

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Objectives
- 1). Read in a classifiable training datset from the Internet using Python for sentiment analysis

- 2). Understand the ethical implications of the dataset I've just identified, both in terms of the legality of its use, and in regards to any inherent biases present in the training data.

- 3). Preprocessing the training dataset to clean unwanted text data, and extract n-grams from the eample documents that can be used for model training.

- 4). Train a logistic regresion model using the cleaned training data that can be used to predict the sentiment of the unclassified examples

- 5). Use a grid search to optimize the logistic regression model.

- 6). Deploy the optimized and trained logistic regression model to the web using Python pickling, and provide a web form for users to further incrementally train your model by entering additional sample data and providing feedback about its classification accuracy.

# Experiment Objective

### Objective 2). Understand the ethical implications of the dataset I've just identified, both in terms of the legality of its use, and in regards to any inherent biases present in the training data. 

- Objective 2 detailed below.

### Describe the dataset I am analyzing, including the TOS and ethical considerations discussed in Lecture 07 & 08 

- The Data set that is being examined is from the University of California - Irvine's Machine Learning repository. The reason that I chose this data set is because it easy to come by. Last project I wasted two days looking for data sets. Now That I know UCI is easy to extract, I just chose one of their data sets so I could spend more time learning the code and less time data engineering a data set into google colab. After going through the excersise I wish I had chosen a data set that had more fields.

- This sources is a well known public data set that is used for machine learning. By choosing to go with a public library I am not worring if I am violating a term of agreement. 

- The Proper cite from the readme is below.
'This dataset was created for the Paper 'From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015'

- https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

### Why did I choose the training examples I did for training my model?

- I chose the amazon data set for the training model for a few reasons. First there was an even amount of positive and negative reviews. Secondly, I feel as though amazon has a good imput for how different products are recieved. This data can have inherent biases from amazong customers in the training data. There might be some reviews that are input by people that simply don't like a supplier. There are other bits that can go into the bias of the training data set such as taking out the stop words and how many Ngrams are used or the combination of the words. Other concerns are how small the data set is.

## How is sentiment determined for the training data?

- Good sentiment is determined with a 1 while bad sentiment is determined with a 0. For the first round I chose to split the data in to 80% training and 20% testing.





# Data Collection

### Am I accesing an API? Scaping a website? Downloading from an archive?

- The data is being downloaded directly from an archive using !weget from google colab. This will grab the data straight from the source and place it straigt on your G-druve. URL below. https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences


### Write the necessary Python code to retrieve and store the training examples to create the model.
- See below

In [ ]:
# Grabbing the websit
!wget 'sentiment_labelled_sentences' 'https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip'

--2022-08-07 09:34:53--  http://sentiment_labelled_sentences/
Resolving sentiment_labelled_sentences (sentiment_labelled_sentences)... failed: Name or service not known.
wget: unable to resolve host address ‘sentiment_labelled_sentences’
--2022-08-07 09:34:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84188 (82K) [application/x-httpd-php]
Saving to: ‘sentiment labelled sentences.zip.8’

sentiment labelled  100%[===================>]  82.21K  --.-KB/s    in 0.1s    

2022-08-07 09:34:54 (835 KB/s) - ‘sentiment labelled sentences.zip.8’ saved [84188/84188]

FINISHED --2022-08-07 09:34:54--
Total wall clock time: 0.4s
Downloaded: 1 files, 82K in 0.1s (835 KB/s)


In [ ]:
# Unzipping the file
!unzip -o 'sentiment labelled sentences.zip'

Archive:  sentiment labelled sentences.zip
  inflating: sentiment labelled sentences/.DS_Store  
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentiment labelled sentences/yelp_labelled.txt  
  inflating: __MACOSX/._sentiment labelled sentences  


### Objective 1). Read in a classifiable training datset from the Internet using Python for sentiment analysis:

### The resulting implementation should be an in-memory pandas dataframe
- Below is the display of the dataframe head

In [ ]:
# Displaying the data in a dataframe
df = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', sep='\t', names=['review', 'Sentiment'])
df.head()

,review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


The Code below is commented out, but was used to push the data set into a csv for later use or for analysis

In [ ]:
# Saving this nice little dataset as a an excel file
from google.colab import files
df.to_csv('amazon_sentiment.csv', encoding = 'utf-8', index=False)
files.download('amazon_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Data Preprocessing

### Objective 3). Preprocessing the training dataset to clean unwanted text data, and extract n-grams from the example documents that can be used for model training.
  - Completed below

### This is where the preprocessing utility for the project will be built.

### Steps required, clean text data, tokenize the document, construct a TfidfVectorizer to be performed here
- Completed below

In [ ]:
# Using the Preprocessing of regular expresions from ch08
def preprocessor(text):
  """This preprocessing function removes all the punctuation, makes the word
  lower case, removews the stop words, and porterStems the words"""
  # Removing all the punctuation
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
  text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')).strip()
  return text

# Note that the tokenizing is set up for a pipeline run. 
def tokenizer(text):
  """Setting up the tokenizer ie split"""
  return text.split()

def tokenizer_porter(text):
  """Tokenizing and porting"""
  porter = PorterStemmer()
  return [porter.stem(word) for word in text.split()]

def tokenizer_stop(text):
  return [word for word in tokenizer_porter(text) if word not in stop]

# constructing a TfidfVectorizer
tfidf = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True)

# Model Optimization and Serialization

### This is where I will use a grid search similar to the one performed in lecture 11: Applying Machine Learning to Sentiment Anlysis

### Find the optimal hyperparameters (including choice of stemming algorithm for TfidVectorizer) to use with my out-of-core capable logistic regression classifier (like SGDClassifier)  Utilizing the TfidVectorizerI created in step three to preprocess my data

- The Code for finding the hyperparameres and the Grid Search are right below

In [ ]:
stop = stopwords.words('english')

# Splitting the data in to x y
x = df['review']
y = df['Sentiment']

# Splitting the data up in to training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)

# Model optimization and serialization
# Setting up the Param Grid
param_grid = [{'vect__ngram_range': [(1,1), (1,2)],
               'vect__stop_words': [stop, None],
               'vect__preprocessor':[preprocessor],
               'vect__tokenizer': [tokenizer, tokenizer_porter, tokenizer_stop],
               'vect__use_idf':[False, True],
               'vect__norm':['l1', 'l2', None],
               'vect__smooth_idf':[True],
               'clf__loss': ['log'],
               'clf__penalty': ['l1','l2'],
               'clf__alpha': [0.001, 0.1, 1.0]},
              {'vect__ngram_range': [(1,1), (1,2)],
               'vect__stop_words': [stop, None],
               'vect__preprocessor':[preprocessor],
               'vect__tokenizer': [tokenizer,tokenizer_porter, tokenizer_stop],
               'vect__use_idf':[False, True],
               'vect__smooth_idf':[True],
               'vect__norm':['l1', 'l2', None],
               'clf__loss': ['log'],
               'clf__penalty': ['l1', 'l2'],
               'clf__alpha': [0.001, 0.1, 1.0]}]

### Objective 4). Train a logistic regresion model using the cleaned training data that can be used to predict the sentiment of the unclassified examples

- Completed Below

### Objective 5). Use a grid search to optimize the logistic regression model.

- Completed below

### Objective: Note the hyperparameters I explore in the grid search must include different values than we used in class. One of the goals of this project is to see if  other options migh yield better results than the book attempted. 

### Objective: Out-of-core learning is a requirement. Since I will be updating my model using the deployed Flask application, the classifier I pickle for use on the website must support incremental learning

- SGDClassifier was used, and this is supported by out-of-core learning

### Objective: The gridsearch I perform wil be against something like an SGDClassifier instance, instead of the LogisticRegression classifier that was used in the book. Because I used the SGDClassifier I changed the value of C for regularisation  to alpha. L1 and L2 are still compared.


- The following paramaters in the param grid have been changed
  - vect__ngram_range: [(1,1)]  to [(1,1),(1,2)]
  - vect__stop_words: Same
  - vect__preprocessor: Has been added
  - vect__tokenizer: [(tokenizer, tokenizer_porter)] to [(tokenizer, tokenizer_porter, tokenizer_stop)]
  - vect__use_idf: [False] to [False, True]
  - vect__smooth_idf: Same
  - vect_norm: [None] to ['l1', 'l2', None]
  - clf__loss: Same [log]
  - clf__plenatly: Same ['l1', 'l2']
  - clf__c: Changed to clf_alpha: values changed to [0.01, 0.1, 1.0]


In [ ]:
# Setting up the Pipeline
pipe = Pipeline(steps=[('vect', tfidf), 
                       ('clf', SGDClassifier(loss='log', random_state=42))])

gs_pipe = GridSearchCV(estimator=pipe, param_grid=param_grid,
                       scoring='accuracy',
                       cv=5,
                       verbose=3,
                       n_jobs=-1,
                       error_score='raise')

gs_pipe.fit(x_train.to_list(), y_train.to_list())

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        SGDClassifier(loss='log',
                                                      random_state=42))]),
             n_jobs=-1,
             param_grid=[{'clf__alpha': [0.001, 0.1, 1.0], 'clf__loss': ['log'],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1), (1, 2)],
                          'vect__norm': ['l1', 'l2', None],
                          'vect__preprocessor': [<function preprocessor a...
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "

In [ ]:
# Displaying the best parameters
print('Hyperparameters parameter set: %s ' % gs_pipe.best_params_)

Hyperparameters parameter set: {'clf__alpha': 0.001, 'clf__loss': 'log', 'clf__penalty': 'l2', 'vect__ngram_range': (1, 2), 'vect__norm': 'l2', 'vect__preprocessor': <function preprocessor at 0x7fd6f65d0440>, 'vect__smooth_idf': True, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7fd6f5f39e60>, 'vect__use_idf': True} 


In [ ]:
print('Train Accuracy: %s ' % gs_pipe.best_score_)

Train Accuracy: 0.8137500000000001 


In [ ]:
print('Test Accuracy: %.3f' % gs_pipe.score(x_test, y_test))

Test Accuracy: 0.830


### Document the results of the grid seach in a markdown cell immediately after the grid search, including computed accuracies against my training and test data sets.

- Train Accuracy: 81.31375%
- Test Accuracy: 83.0%

The hyper parameters that were chosen are:

- For the Vect
  - vect__ngram_range: (1,2)
  - vect__norm: l2
  - vect__preprocessor: Preprocessor
  - vect__stop_words: None
  - vect__tokenizer: tokenizer_porter
  - vect__use_idf: True
  - vect__smooth_idf: True

- For CLF
  - clf_loss: log
  - clf__alpha: 0.01
  - clf__penalty: l2


### Be sure to exaplain in the write up for this section why I chose the values that I did for the hyperparameters, and if they performed better for my data set than the other ones the booke attempted to use

- When comparing the train and test accuraries the test actually performed better. 
- For Ngrams I decided to look at both (1,1) and (1,2). (1,1) only uses unigrams where as (1,2) uses both unigrams and bigrams. Unsprisingly the (1,2) helped the model perform better. this is using combinations of words to help determine the sentiment.
- vect__norm: Was set to l2 indicating that regularization that is standard performed bessed.
- Preprocessor was used in this case so that the pickling could be conducted easier with out having to clean the data set. This way when we go back to update the model it will always clean the data.
- The tokenizer that was used was tokenizer_porter. This tokenizer used stop words and also stemmed the words. 
- vect__stop_words: This came up as using none in the parameters, This maybe because of the Ngrams using combinations of two words.
- Use IDF had the choice of both False or True. In this case True was chosen as making the model better. This means that inverse document frequency reweighting is used.
- Vect__smooth: Is unsuprisingly True
- When looking at the SGDClassifer the alpha that was chosen was .001. A higher alpha means the migher the regulaization. In this case the least amount of regulaization was chosen.
- For the SGDClassifer the best parameter was L2 which is the standard regulizer for linear SVM models. L1 would have brought more sparity to the model


### After documenting the findings above. I am fitting the classifer against the entire data set and then pickling the resulting object to a file that will be used in the website.

* Note: The grid search I perform agaisnt the SGDClassifier instance, instead of the LogisticRegrssion classifier that was used in the book if you use SFDClassifer, instead of varying the value of C for regularization, you will vary the values for alpha. Still compare l1 vs l2 regulatization. This will result in a comparison of my work to the example in the book that isnt' strictly 1-1 but will be close enough to draw meaningful conculsions against

In [ ]:
# Some of the code below is re-written to make it easier for the creation of the vectorizer
stop = stopwords.words('english')
porter = PorterStemmer()

def preprocessor(text):
  """This preprocessing function removes all the punctuation"""
  # Removing all the punctuation
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
  text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')).strip()
  return text

# Note that the tokenizing is set up for a pipeline run. 
def tokenizer_porter(text):
  """Tokenizing and porting"""
  return [porter.stem(word) for word in text.split()]

# Creating a function for streaming
def stream_docs(path):
  with open(path, 'r', encoding='utf-8', newline='\n') as csv:
    next(csv) # skipping the header
    for line in csv:
      text, label = line[:-3], line[:-1][-1]
      yield text, label

def get_minibatch(doc_stream, size):
  docs, y = [], []
  try:
    for _ in range(size):
      text, label = next(doc_stream)
      docs.append(text)
      y.append(int(label))
  except StopIteration:
    return None, None
  return docs, y

# Setting the Hashing Vectorize with the params from above
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2*22,
                         norm='l2',
                         ngram_range=(1,2),
                         preprocessor=preprocessor,
                         tokenizer=tokenizer_porter,
                         stop_words=stop)

# Setting the SGDClassifier with the parms from above
clf = SGDClassifier(alpha=0.01, loss='log', penalty='l2', random_state=42)

In [ ]:
# # Setting up Google drive to take in the Dataframe that was extracted straight from the website
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('content/')
path = 'amazon_sentiment.csv'

with open(path, 'w', encoding = 'utf-8') as f:
  df.to_csv(f)

Mounted at content/


In [ ]:
#  Showing an example of the mount working
next(stream_docs('amazon_sentiment.csv'))

('0,So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

In [ ]:
doc_stream = stream_docs(path='amazon_sentiment.csv')

In [ ]:
import pyprind
pbar = pyprind.ProgBar(10)
for _ in range(10):
  x_train, y_train = get_minibatch(doc_stream, size=100)
  if not x_train:
    break
  x_train = vect.transform(x_train)
  clf.partial_fit(x_train, y_train, classes=np.array([1,0]))
  pbar.update()

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  % sorted(inconsistent)
0% [##########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
print('Accuracy: %.3f' % clf.score(x_train, y_train))

Accuracy: 0.630


Note that when using the minibatching to train the model that the the accuracy went down and is now 63-64% (depending on the run) this is a notable difference from above. But it does show that the vectorizer can be used with out of core learning. This will be what we use in the standalone python file.

### Creating the Pickle
- Below I will create a pickle on the full data set. Using the out of core model. Though this is not as accurate as using TfidVectorizer it is a requirement for the project.

In [ ]:
x_train = df['review'].values
y_train = df['Sentiment'].values

x_train = vect.transform(x_train)
clf.fit(x_train, y_train)
print('Accuracy: %.3f' % clf.score(x_train, y_train))

Accuracy: 0.648


Once again when training the full dataset using the Hashingvectorizer the accuracy is a bit lower than we would have like than just using the TfidVectorizer. But we'll take it as working and move on toward pickling.

In [ ]:
import pickle
# drive.flush_and_mount()
# drive.flush_and_unmount()
# drive.mount('content/', force_remount=True)

In [ ]:
# Changing the google drive mount
%cd content/MyDrive/ML_Colab/Project_2

/content/content/MyDrive/ML_Colab/Project_2


In [ ]:
# Directory already made
# %mkdir pkl_objects

In [ ]:
# os.path
!find

.
./Project_02_Duncan_Ferguson_v2.ipynb
./__pycache__
./__pycache__/vectorizer.cpython-37.pyc
./pkl_objects
./pkl_objects/classifier.pkl
./pkl_objects/stopwords.pkl
./static
./static/style.css
./templates
./templates/reviewform.html
./templates/results.html
./templates/thanks.html
./templates/_formhelpers.html
./sentiment labelled sentences.zip
./sentiment labelled sentences
./sentiment labelled sentences/.DS_Store
./sentiment labelled sentences/amazon_cells_labelled.txt
./sentiment labelled sentences/imdb_labelled.txt
./sentiment labelled sentences/readme.txt
./sentiment labelled sentences/yelp_labelled.txt
./__MACOSX
./__MACOSX/sentiment labelled sentences
./__MACOSX/sentiment labelled sentences/._.DS_Store
./__MACOSX/sentiment labelled sentences/._imdb_labelled.txt
./__MACOSX/sentiment labelled sentences/._readme.txt
./__MACOSX/._sentiment labelled sentences
./amazon_reviews.db
./update.py
./vectorizer.py
./amazon_app.py
./Copy of Project_02_Duncan_Ferguson_v2.ipynb


In [ ]:
# Showing the Directory before making the pickle
os.listdir()

['Project_02_Duncan_Ferguson_v2.ipynb',
 '__pycache__',
 'pkl_objects',
 'static',
 'templates',
 'sentiment labelled sentences.zip',
 'sentiment labelled sentences',
 '__MACOSX',
 'amazon_reviews.db',
 'update.py',
 'vectorizer.py',
 'amazon_app.py',
 'Copy of Project_02_Duncan_Ferguson_v2.ipynb']

In [ ]:
%cd pkl_objects/

/content/content/MyDrive/ML_Colab/Project_2/pkl_objects


In [ ]:
import pickle
# Making my two pickles... Stop words was not chosen by the hyper parama
pickle.dump(stop, open('stopwords.pkl', 'wb'), protocol=4)
pickle.dump(clf, open('classifier.pkl', 'wb'), protocol=4)

In [ ]:
# Showing the directory after making the pickle
# Displaying that the pickles have been made
display(os.getcwd())
display(os.listdir())

'/content/content/MyDrive/ML_Colab/Project_2/pkl_objects'

['classifier.pkl', 'stopwords.pkl']

In [ ]:
%cd ..

/content/content/MyDrive/ML_Colab/Project_2


# Creat a standalone Python file

### Objective: File should include all of the data preprocessing functionality created in step 4 named vectorizer.py. 
### Objective: The creation of vectorizer.py should be done inside project02.pynb file similiar to ch09.ipynb

In [ ]:
# Displaying that a vectorizer.py file has been created in the file
display(os.getcwd())
display(os.listdir())

'/content/content/MyDrive/ML_Colab/Project_2'

['Project_02_Duncan_Ferguson_v2.ipynb',
 '__pycache__',
 'pkl_objects',
 'static',
 'templates',
 'sentiment labelled sentences.zip',
 'sentiment labelled sentences',
 '__MACOSX',
 'amazon_reviews.db',
 'update.py',
 'vectorizer.py',
 'amazon_app.py',
 'Copy of Project_02_Duncan_Ferguson_v2.ipynb']

### Creating the other necessary folders
- Making static folder
- Making templates folder


In [ ]:
# Directories already made
# %mkdir static
# %mkdir templates

In [ ]:
display(os.getcwd())
display(os.listdir())

'/content/content/MyDrive/ML_Colab/Project_2'

['Project_02_Duncan_Ferguson_v2.ipynb',
 '__pycache__',
 'pkl_objects',
 'static',
 'templates',
 'sentiment labelled sentences.zip',
 'sentiment labelled sentences',
 '__MACOSX',
 'amazon_reviews.db',
 'update.py',
 'vectorizer.py',
 'amazon_app.py',
 'Copy of Project_02_Duncan_Ferguson_v2.ipynb']

### Objective: This file will initialize a HashingVectorizer using the parameters I learned tuning TfidVectorizer in step Four
- When you look in the file, you will find the same as created from the hashing vectorizer above.

# Website Creation and Publishing

### Objective: Create an interactive website using Flask, SQlite, WTforms, and Jinja2, that will enable me to test the accuracy of my calculated model by prompting users to enter text in the same form as your train data (product reviews, social media posts, etc. ) and perform sentiment analysis against their submissionts

- Link to the working! Webapp: http://duncanferguson.pythonanywhere.com/

In [ ]:
os.getcwd()

'/content/content/MyDrive/ML_Colab/Project_2'

In [ ]:
os.listdir()

['Project_02_Duncan_Ferguson_v2.ipynb',
 '__pycache__',
 'pkl_objects',
 'static',
 'templates',
 'sentiment labelled sentences.zip',
 'sentiment labelled sentences',
 '__MACOSX',
 'amazon_reviews.db',
 'update.py',
 'vectorizer.py',
 'amazon_app.py',
 'Copy of Project_02_Duncan_Ferguson_v2.ipynb']


### Objective: Very similar to Lecture 12 -Embedding a Machine Learning Model into a Web Application, I will then show the user my sentiment prediction base on their entry, allowing them to rate whether the prediction was accurate or not. 

- Completed: http://duncanferguson.pythonanywhere.com/



### Objective: User provided examples will be stored in SQLite and be used to incrementally refine the trained model I created in step 4. 



In [ ]:
import sqlite3

conn = sqlite3.connect('amazon_reviews.db')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS amazon_review_db')
c.execute('CREATE TABLE amazon_review_db (review TEXT, sentiment INTEGER)')

example1 = 'This product is horrible!'
c.execute("INSERT INTO amazon_review_db (review, sentiment) VALUES (?, ?)", (example1,0))

example2 = 'Testing 1, Testing 2 Testing, This product should do the trick'
c.execute("INSERT INTO amazon_review_db (review, sentiment) VALUES (?, ?)", (example2,1))

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('amazon_reviews.db')
c = conn.cursor()
c.execute('Select * FROM amazon_review_db')
results = c.fetchall()
conn.close()

In [ ]:
print(results)

[('This product is horrible!', 0), ('Testing 1, Testing 2 Testing, This product should do the trick', 1)]


### Objective: Complete website will be a published to PthonAnywhere and a link to the website will be included in a markdown cell in this Jupter notebook.

- Completed: http://duncanferguson.pythonanywhere.com/



### Objective: The source code for the website (including Python source code, HTML templates, and Python pickle files) will be submitted alongside the Jupyter Notbook in Goodle Drive 



In [ ]:
os.getcwd()
os.listdir()

['Project_02_Duncan_Ferguson_v2.ipynb',
 '__pycache__',
 'pkl_objects',
 'static',
 'templates',
 'sentiment labelled sentences.zip',
 'sentiment labelled sentences',
 '__MACOSX',
 'amazon_reviews.db',
 'update.py',
 'vectorizer.py',
 'amazon_app.py',
 'Copy of Project_02_Duncan_Ferguson_v2.ipynb']

### Objective Note: I must use the pickled model created in step four as the starting point for my classifier. In addition, if your webserver was to be restared, all user-provided examples must still represet in the trained model (i.e. I cannot rest the pickled version only on server restart)

- The pickles from above were used. They were done in such a way that if the webserver was restated all user-provided examples would still be represent in the trained model.



### Objective Note: Make sure to look closely at the code thats being reused for the Flask application to make sure that assumptions built into the existing app still hold for the dataset. For instance, make sure that the handling of correct/incorrect feedback via the website works for my data.

- Completed. Reused flask application is work and they are handling the correct/incorrect feedback properly.

http://duncanferguson.pythonanywhere.com/